### Analysis of Behavioral Risk Factors Involved in Motorcycle Fatalities


In [53]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact


In [54]:
df = pd.read_csv('CRSS_1619_csv.csv',encoding='cp1252')
df.head()

,CASENUM,VEH_NO,MONTH,HOUR,YEAR,DAY_WEEK,INJURY,REST_USENAME,HELMET,DRINKING,DRUGS,IMPAIRMENT
0,2.016000e+11,1,1,17,2016,2,Injury,No Helmet,False,False,False,None
1,2.016000e+11,1,1,19,2016,1,Injury,Helmet,True,True,False,"Drugs, Alcohol, or Medication"
2,2.016000e+11,2,1,12,2016,7,Injury,Helmet,True,False,False,None
3,2.016000e+11,2,1,3,2016,2,Injury,Helmet,True,False,False,NaN
4,2.016000e+11,1,1,16,2016,5,Injury,No Helmet,False,False,False,None


In [55]:
#drop all rows with null values
df = df.dropna(axis=0, inplace=False)

In [56]:
#Chech value counts for injuries
df['INJURY'].value_counts()

Injury          7361
No Injury        449
Fatal Injury     225
Name: INJURY, dtype: int64

In [57]:
#Create dataframe with fatal and not fatal injuries
df_fatal= df.copy()
df_fatal['INJURY'] =df_fatal['INJURY'].replace(['No Injury'], 'Not Fatal')
df_fatal['INJURY'] = df_fatal['INJURY'].replace(['Injury'], 'Not Fatal')
df_fatal.INJURY.value_counts()

Not Fatal       7810
Fatal Injury     225
Name: INJURY, dtype: int64

In [58]:
#create binary columns indicating whether behavioral risk factors were present or not
df_fatal[['INJURY', 'NO_INJURY']] = pd.get_dummies(df_fatal['INJURY'])
df_fatal[['NOT_DRINKING','DRINKING']] = pd.get_dummies(df_fatal['DRINKING'])
df_fatal[['NO_DRUGS','DRUGS']] = pd.get_dummies(df_fatal['DRUGS'])
df_fatal[['NO_HELMET', 'HELMET']] = pd.get_dummies(df_fatal['HELMET'])
df_fatal.head()

,CASENUM,VEH_NO,MONTH,HOUR,YEAR,DAY_WEEK,INJURY,REST_USENAME,HELMET,DRINKING,DRUGS,IMPAIRMENT,NO_INJURY,NOT_DRINKING,NO_DRUGS,NO_HELMET
0,2.016000e+11,1,1,17,2016,2,0,No Helmet,0,0,0,None,1,1,1,1
1,2.016000e+11,1,1,19,2016,1,0,Helmet,1,1,0,"Drugs, Alcohol, or Medication",1,0,1,0
2,2.016000e+11,2,1,12,2016,7,0,Helmet,1,0,0,None,1,1,1,0
4,2.016000e+11,1,1,16,2016,5,0,No Helmet,0,0,0,None,1,1,1,1
5,2.016000e+11,1,1,11,2016,2,0,Helmet,1,0,1,None,1,1,0,0


In [59]:
#Drop all unnecessary columns 
dropcolumns=['CASENUM', 'VEH_NO', 'MONTH', 'HOUR', 'DAY_WEEK', 'REST_USENAME',
             'NO_INJURY', 'NOT_DRINKING', 'NO_DRUGS', 'IMPAIRMENT', 'HELMET']

In [60]:
df_drinkdrug = df_fatal.drop(dropcolumns, axis=1)
df_drinkdrug.head()

,YEAR,INJURY,DRINKING,DRUGS,NO_HELMET
0,2016,0,0,0,1
1,2016,0,1,0,0
2,2016,0,0,0,0
4,2016,0,0,0,1
5,2016,0,0,1,0


In [61]:
import statsmodels.formula.api as smf

In [62]:
log_reg = smf.logit("INJURY ~ DRINKING + DRUGS + NO_HELMET", data=df_drinkdrug).fit()
log_reg.summary()

Optimization terminated successfully.
         Current function value: 0.122498
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                 8035
Model:                          Logit   Df Residuals:                     8031
Method:                           MLE   Df Model:                            3
Date:                Mon, 19 Dec 2022   Pseudo R-squ.:                 0.04095
Time:                        09:52:45   Log-Likelihood:                -984.27
converged:                       True   LL-Null:                       -1026.3
Covariance Type:            nonrobust   LLR p-value:                 4.146e-18
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.8531      0.091    -42.304      0.000      -4.032      -3.675
DRINKING       0.8918      0.220      4.046      0.000       0.460       1.324
DRUGS          1.5987      0.303      5.279      0.000       1.005       2.192
NO_HELMET      0.4437      0.142      3.134      0.002       0.166       0.721
==============================================================================
"""

In [63]:
#confidence intevals 
log_reg.conf_int(0.05)

,0,1
Intercept,-4.031600,-3.674571
DRINKING,0.459764,1.323774
DRUGS,1.005142,2.192203
NO_HELMET,0.166214,0.721151


In [64]:
log_reg.pvalues

Intercept    0.000000e+00
DRINKING     5.212975e-05
DRUGS        1.297798e-07
NO_HELMET    1.724069e-03
dtype: float64

### The Hypothesis
> H<sub>0</sub>: Behavioral risk factors have no impact on motorcycle fatalities   
> H<sub>1</sub>: Behavioral risk factors have an impact on motorcycle fatalities 


In [65]:
p_values = log_reg.pvalues
n = 0
for i in p_values:
    
    alpha = 0.05
    index = p_values.index[n]
    print("p value is of {}: {}".format(index,i))
    if i <= alpha:
        print('Dependent (reject H0)')
    else:
        print('Independent (H0 holds true)')
    n +=1


p value is of Intercept: 0.0
Dependent (reject H0)
p value is of DRINKING: 5.212975494559715e-05
Dependent (reject H0)
p value is of DRUGS: 1.2977979461201601e-07
Dependent (reject H0)
p value is of NO_HELMET: 0.0017240691393289394
Dependent (reject H0)


In [66]:
#the regression coefficient (b1) is the estimated increase in the log odds of the outcome per unit increase
#in the value of the exposure (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2938757/)
# ... Define and fit model
odds_ratios = pd.DataFrame(
    {
        "Odds Ratio": log_reg.params,
        "Lower CI": log_reg.conf_int()[0],
        "Upper CI": log_reg.conf_int()[1],
    }
)
odds_ratios = np.exp(odds_ratios)
print(odds_ratios)

           Odds Ratio  Lower CI  Upper CI
Intercept    0.021214  0.017746  0.025360
DRINKING     2.439441  1.583700  3.757575
DRUGS        4.946463  2.732296  8.954923
NO_HELMET    1.558436  1.180826  2.056799


### Odds Ratio
With a 95% confidence interval:
> people who drank alcohol were between 1.5 and 3.75 times more likely die in a motocycle accident    
> people who engaged in drug use were between 2.73 and 8.95 times more likely die in a motocycle accident    
> people who did not use a helmet were between 1.18 and 2.05 times more likely die in a motocycle accident  


In [67]:
log_reg.params

Intercept   -3.853086
DRINKING     0.891769
DRUGS        1.598673
NO_HELMET    0.443683
dtype: float64

#### Chi-square test will be applied to each behavior to see if there is an impact on the rate of injury



> H<sub>0</sub>: Drinking has no impact on motorcycle fatalities   
> H<sub>1</sub>: Drinking has an impact on motorcycle fatalities 

In [68]:
#first the data is converted to a contingency table with frequencies
contigency= pd.crosstab(df_drinkdrug['INJURY'], df_drinkdrug['DRINKING']) 
contigency

DRINKING,0,1
INJURY,,
0,7407,403
1,183,42


In [69]:
c, p, dof, expected = chi2_contingency(contigency, correction=True) 
expected

array([[7377.46110765,  432.53889235],
       [ 212.53889235,   12.46110765]])

> Because all of the expected results are greater than five the chi-square test can be trusted. 

In [70]:
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 9.082238135917225e-18
Dependent (reject H0)


> The p-value is lower than .05 so the null hypothesis is rejected and it is evident that drinking has an impact on motorcycle fatalities

> H<sub>0</sub>: Helmet use has no impact on motorcycle fatalities   
> H<sub>1</sub>: Helemt use has an impact on motorcycle fatalities 


In [71]:
helcontigency= pd.crosstab(df_drinkdrug['INJURY'], df_drinkdrug['NO_HELMET']) 
helcontigency

NO_HELMET,0,1
INJURY,,
0,5609,2201
1,132,93


In [72]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(helcontigency) 
# Print the expected values
expected

array([[5580.23771002, 2229.76228998],
       [ 160.76228998,   64.23771002]])

> Because all of the expected results are greater than five the chi-square test can be trusted. 

In [73]:
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 2.3229324874636095e-05
Dependent (reject H0)


> The p-value is lower than .05 so the null hypothesis is rejected and it is evident that helmet use has an impact on motorcycle fatalities

> H<sub>0</sub>: Drug use has no impact on motorcycle fatalities   
> H<sub>1</sub>: Drug use has an impact on motorcycle fatalities 

In [74]:
contigency_drug= pd.crosstab(df_drinkdrug['INJURY'], df_drinkdrug['DRUGS']) 
contigency_drug

DRUGS,0,1
INJURY,,
0,7731,79
1,203,22


In [75]:
# Chi-square test of independence. 
c, p, dof, expected = chi2_contingency(contigency_drug) 
# Print the expected values
expected

array([[7.71182825e+03, 9.81717486e+01],
       [2.22171749e+02, 2.82825140e+00]])

> Because all of the expected results are not greater than five I am unsure if the chi-square test can be trusted and will run a fisher test because it is better for smaller sized samples. 

In [76]:
#Use fisher test because the size of observations is small  
#Chi-square test of independence. 
oddr, p = fisher_exact(contigency_drug) 
# Print the expected values
oddr, p


(10.605599551038225, 3.2960302705117844e-14)

In [77]:
# interpret p-value
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (H0 holds true)')

p value is 3.2960302705117844e-14
Dependent (reject H0)


> The p-value is lower than .05 so the null hypothesis is rejected and it is evident that drug use has an impact on motorcycle fatalities

### Conclusion:

> Based on these findings, we can conclude that behavioral risk factors significantly impact motorcycle fatalities. Therefore, it is in the best interest of all involved to proceed with the creation of educational materials based on these findings. Most importantly, the issue of drug use should be addressed first as it has the most consequential impact on the likelihood of death. 